In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from utils import label_accuracy_score, get_miou, FocalTverskyLoss
import cv2

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

# seed 고정
random_seed = 21
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

pytorch version: 1.7.1
GPU 사용 가능 여부: True
GeForce RTX 3070
1


In [4]:
%matplotlib inline

dataset_path = '/opt/ml/input/data'
anns_file_path = dataset_path + '/' + 'train.json'

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)

# Load categories and super categories
cat_names = [] # 모든 카테고리 이름
for cat_it in categories:
    cat_names.append(cat_it['name'])

print('Number of categories:', nr_cats)
print('Number of annotations:', nr_annotations)
print('Number of images:', nr_images)

# Count annotations
cat_histogram = np.zeros(nr_cats, dtype=int)
for ann in anns:
    cat_histogram[ann['category_id']] += 1 # 카테고리별 개수 카운트

# Convert to DataFrame
df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})

# category labeling 
sorted_temp_df = df.sort_index()

# background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
sorted_df = pd.DataFrame(["Backgroud"], columns = ["Categories"])
sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)

# class (Categories) 에 따른 index 확인 (0~11 : 총 12개)
# sorted_df

category_names = list(sorted_df.Categories)

def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id'] == classID:
            return cats[i]['name']
    return "None"

Number of categories: 11
Number of annotations: 21116
Number of images: 2617


In [6]:
exp_title = "TESTING" # 실험 이름
batch_size = 2   # Mini-batch size
num_epochs = 2
learning_rate = 0.001

In [7]:
class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
#         images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((12, image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks[pixel_value] = self.coco.annToMask(anns[i])
                
            masks = masks.astype(np.float32)

            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

## Dataset 정의 및 DataLoader 할당

In [8]:
# train.json / validation.json / test.json 디렉토리 설정
train_path = dataset_path + '/train.json'
val_path = dataset_path + '/val.json'

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

train_transform = A.Compose([
                            A.Normalize(mean=[0.4611, 0.4403, 0.4193], std=[0.2107, 0.2074, 0.2157]),
                            ToTensorV2()
                            ])

val_transform = A.Compose([
                          A.Normalize(mean=[0.4611, 0.4403, 0.4193], std=[0.2107, 0.2074, 0.2157]),
                          ToTensorV2()
                          ])

# create own Dataset 1 (skip)
# validation set을 직접 나누고 싶은 경우
# random_split 사용하여 data set을 8:2 로 분할
# train_size = int(0.8*len(dataset))
# val_size = int(len(dataset)-train_size)
# dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=transform)
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# create own Dataset 2
# train dataset
train_dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=train_transform)

# validation dataset
val_dataset = CustomDataLoader(data_dir=val_path, mode='val', transform=val_transform)


# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=0,
                                           collate_fn=collate_fn)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=0,
                                         collate_fn=collate_fn)


loading annotations into memory...
Done (t=3.89s)
creating index...
index created!
loading annotations into memory...
Done (t=0.87s)
creating index...
index created!


## baseline model

### FCN8s (VGG imageNet weight)

In [9]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

In [10]:
import segmentation_models_pytorch as smp

In [11]:
model = smp.DeepLabV3Plus(
    encoder_name="resnet101",
    classes=12,
)
model = model.to(device)

In [12]:
x = torch.randn([2, 3, 512, 512]).to(device)
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())

input shape :  torch.Size([2, 3, 512, 512])
output shape :  torch.Size([2, 12, 512, 512])


In [12]:
criterion = FocalTverskyLoss()

In [17]:
cel = nn.CrossEntropyLoss()

In [13]:
for images, masks, _ in train_loader:
    images = torch.stack(images)       # (batch, channel, height, width)
    masks = torch.stack(masks).long()  # (batch, channel, height, width)
    outputs = model(images.to(device))
    
    print("loss try")
    loss = criterion(outputs, masks.to(device))
    print(loss.item())
   

loss try
0.7742109298706055
loss try
0.9698048830032349
loss try
0.984403133392334
loss try
0.7326148748397827
loss try
0.7645780444145203
loss try
0.9400026798248291
loss try
1.1154944896697998
loss try
0.5611982345581055
loss try
0.8945510387420654
loss try
0.8198568820953369


KeyboardInterrupt: 

In [15]:
loss

tensor(0.8100, device='cuda:0', grad_fn=<PowBackward0>)

In [16]:
loss.item()

0.8100382089614868

In [17]:
loss.backward()

In [18]:
loss.item()

0.8100382089614868

## train, validation, test 함수 정의

In [16]:
def train(num_epochs, model, data_loader, val_loader, criterion, optimizer, scheduler, saved_dir, val_every, device):
    print('Start training...')
    best_miou = 0.
    for epoch in range(num_epochs):
        model.train()
        for step, (images, masks, _) in enumerate(data_loader):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)
            
            
            
            # gpu 연산을 위해 device 할당
            images, masks = images.to(device), masks.to(device)
                  
            # inference
            outputs = model(images)
            
            # loss 계산 (cross entropy loss)
            loss = criterion(outputs, masks)
            
            # loss = focal_criterion(outputs, masks)
            # loss = (ce_loss + focal_loss) / 2

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
                   
            # wandb
            wandb.log({'train_loss': loss.item(), 'train_step': step+1})
            
            # step 주기에 따른 loss 출력
            if (step + 1) % 25 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                    epoch+1, num_epochs, step+1, len(train_loader), loss.item()))
        
        # validation 주기에 따른 loss 출력 및 best model 저장
        if (epoch + 1) % val_every == 0:
            avrg_loss, miou = validation(epoch + 1, model, val_loader, criterion, device)
            wandb.log({'learning_rate': get_lr(optimizer), 'epoch': epoch+1})
            scheduler.step(1 - miou)
            
            if miou > best_miou:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_miou = miou
                save_model(model, saved_dir)

In [17]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch))
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        mIoU_list = []
        for step, (images, masks, _) in enumerate(data_loader):
            
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)

            images, masks = images.to(device), masks.to(device)            

            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            outputs = torch.argmax(outputs.squeeze(), dim=1).detach().cpu().numpy()

            mIoU = get_miou(masks.detach().cpu().numpy(), outputs, n_class=12)
            mIoU_list.extend(mIoU)
            
        avrg_loss = total_loss / cnt
        miou = np.mean(mIoU_list)
        print('Validation #{}  Average Loss: {:.4f}, mIoU: {:.4f}'.format(epoch, avrg_loss, np.mean(mIoU_list)))
        wandb.log({'val_loss': avrg_loss, 'mIoU': miou, 'epoch': epoch})
        
    return avrg_loss, miou

## 모델 저장 함수 정의

In [18]:
# 모델 저장 함수 정의
val_every = 1 

saved_dir = './saved'
if not os.path.isdir(saved_dir):                                                           
    os.mkdir(saved_dir)
    
def save_model(model, saved_dir, file_name=f'{exp_title}.pt'):
    check_point = {'net': model.state_dict()}
    output_path = os.path.join(saved_dir, file_name)
    torch.save(model.state_dict(), output_path)

## 모델 생성 및 Loss function, Optimizer 정의

In [1]:
from loss import FocalLoss


In [20]:
# Loss function 정의
# loss_weights = torch.ones(12)
# loss_weights[0] = 0.1
# criterion = nn.CrossEntropyLoss(weight=loss_weights.to(device))

In [21]:
ce_criterion = nn.CrossEntropyLoss()
focal_criterion = FocalLoss()
criterioni = FocalTverskyLoss()

# Optimizer 정의
optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate, weight_decay=1e-6)

# Scheduler
scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=1)
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
train(num_epochs, model, train_loader, val_loader, ce_criterion, focal_criterion, optimizer, scheduler, saved_dir, val_every, device)

Start training...
Epoch [1/30], Step [25/164], Loss: 1.3214
Epoch [1/30], Step [50/164], Loss: 0.9597
Epoch [1/30], Step [75/164], Loss: 0.7018
Epoch [1/30], Step [100/164], Loss: 0.6356
Epoch [1/30], Step [125/164], Loss: 0.4910
Epoch [1/30], Step [150/164], Loss: 0.6206
Start validation #1
Validation #1  Average Loss: 0.4822, mIoU: 0.4872
Best performance at epoch: 1
Save model in ./saved
Epoch [2/30], Step [25/164], Loss: 0.5862
Epoch [2/30], Step [50/164], Loss: 0.3455
Epoch [2/30], Step [75/164], Loss: 0.4136
Epoch [2/30], Step [100/164], Loss: 0.5503
Epoch [2/30], Step [125/164], Loss: 0.3545
Epoch [2/30], Step [150/164], Loss: 0.5782
Start validation #2
Validation #2  Average Loss: 0.3688, mIoU: 0.4983
Best performance at epoch: 2
Save model in ./saved
Epoch [3/30], Step [25/164], Loss: 0.2385
Epoch [3/30], Step [50/164], Loss: 0.3025
Epoch [3/30], Step [75/164], Loss: 0.3416
Epoch [3/30], Step [100/164], Loss: 0.3690
Epoch [3/30], Step [125/164], Loss: 0.3004
Epoch [3/30], Step

In [ ]:
# 훈련 완료, wandb 종료
print("training complete!")
wandb_run.finish()
del train_dataset, val_dataset, train_loader, val_loader

## 저장된 model 불러오기 (학습된 이후) 

In [ ]:
# best model 저장된 경로
model_path = f'./saved/{exp_title}.pt'

# best model 불러오기
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)

# 추론을 실행하기 전에는 반드시 설정 (batch normalization, dropout 를 평가 모드로 설정)
# model.eval()

In [ ]:
# test dataset
test_path = dataset_path + '/test.json'
test_transform = A.Compose([
    A.Normalize(mean=[0.4611, 0.4403, 0.4193], std=[0.2107, 0.2074, 0.2157]),
    ToTensorV2()])
test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          num_workers=4,
                                          collate_fn=collate_fn)

## submission을 위한 test 함수 정의

In [ ]:
def test(model, data_loader, device):
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    model.eval()
    
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(test_loader):

            # inference (512 x 512)
            outs = model(torch.stack(imgs).to(device))
            oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

## submission.csv 생성

In [ ]:
# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/code/submission/sample_submission.csv', index_col=None)

# test set에 대한 prediction
file_names, preds = test(model, test_loader, device)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(f"/opt/ml/code/submission/{exp_title}.csv", index=False)

In [ ]:
print("BYE!")